<a href="https://colab.research.google.com/github/Arshad221b/Smart-Store/blob/master/faceexpression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd

In [0]:
download = drive.CreateFile({'id': '1PMCsXW9ks0GblAddXTzHLPk7X2kDQwm0'})
download.GetContentFile('fer2013.csv')
data = pd.read_csv('fer2013.csv')

In [0]:
data2 = data['Usage'] == 'Training'

In [0]:
train = data[data2]

In [0]:
data3 = data['Usage'] == 'PrivateTest'

In [0]:
test  = data[data3]

In [0]:
y_train = train['emotion'].values
y_test  = test['emotion'].values

In [0]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)


In [0]:
y_test = to_categorical(y_test)

In [0]:
X_train = train['pixels'].str.split(' ', expand=True).rename(columns = lambda x: "pixel"+str(x+1))

In [0]:
X_test = test['pixels'].str.split(' ', expand=True).rename(columns = lambda x: "pixel"+str(x+1))

In [33]:
print(X_train.shape)

(28709, 2304)


In [36]:
print(X_test.shape)

(3589, 2304)


In [0]:
X_train = X_train.values.reshape((28709, 48, 48, 1))
X_test = X_test.values.reshape((3589 , 48 , 48 , 1))

In [0]:
model = Sequential()
 
#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))
 
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
model.add(Flatten())
 
#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
 
model.add(Dense(7, activation='softmax'))


In [48]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=256)


Instructions for updating:
Use tf.cast instead.
Epoch 1/20
28709/28709 [==============================] - 32s 1ms/step - loss: 1.8577 - acc: 0.2515
Epoch 2/20
28709/28709 [==============================] - 25s 880us/step - loss: 1.6546 - acc: 0.3402
Epoch 3/20
28709/28709 [==============================] - 26s 908us/step - loss: 1.4992 - acc: 0.4199
Epoch 4/20
28709/28709 [==============================] - 26s 910us/step - loss: 1.4065 - acc: 0.4619
Epoch 5/20
28709/28709 [==============================] - 26s 912us/step - loss: 1.3193 - acc: 0.4991
Epoch 6/20
28709/28709 [==============================] - 26s 913us/step - loss: 1.2534 - acc: 0.5260
Epoch 7/20
28709/28709 [==============================] - 26s 910us/step - loss: 1.2090 - acc: 0.5423
Epoch 8/20
28709/28709 [==============================] - 26s 912us/step - loss: 1.1543 - acc: 0.5631
Epoch 9/20
28709/28709 [==============================] - 26s 915us/step - loss: 1.1102 - acc: 0.5809
Epoch 10/20
28709/28709 [===========

In [49]:
accuracy = model.evaluate(x=X_test,y=y_test,batch_size=256)
print("Accuracy: ",accuracy[1])

3589/3589 [==============================] - 2s 485us/step
Accuracy:  0.5561437726469218


In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
average_pooling2d_5 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 5, 5, 128)         73856     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 3, 3, 128)         147584    
__________

In [52]:
model.save('Facemodel.h5')
weights_file = drive.CreateFile({'title' : 'Facemodel.h5'})
weights_file.SetContentFile('Facemodel.h5')
weights_file.Upload()
drive.CreateFile({'id': weights_file.get('id')})

GoogleDriveFile({'id': '1_-wInEGwuQBaPBvo1M1t8Yys0eZjLbbT'})